In [1]:
import pandas as pd
import numpy as np
from geopy.distance import geodesic
import math

#### Bringing in all the datasets

In [2]:
races = pd.read_csv('../data/nyra_race_table.csv')
races.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   track_id         2000 non-null   object
 1   race_date        2000 non-null   object
 2   race_number      2000 non-null   int64 
 3   distance_id      2000 non-null   int64 
 4   course_type      2000 non-null   object
 5   track_condition  2000 non-null   object
 6   run_up_distance  2000 non-null   int64 
 7   race_type        2000 non-null   object
 8   purse            2000 non-null   int64 
 9   post_time        2000 non-null   int64 
dtypes: int64(5), object(5)
memory usage: 156.4+ KB


In [3]:
starts = pd.read_csv('../data/nyra_start_table.csv')
starts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14915 entries, 0 to 14914
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   track_id        14915 non-null  object
 1   race_date       14915 non-null  object
 2   race_number     14915 non-null  int64 
 3   program_number  14915 non-null  object
 4   weight_carried  14915 non-null  int64 
 5   jockey          14915 non-null  object
 6   odds            14915 non-null  int64 
dtypes: int64(3), object(4)
memory usage: 815.8+ KB


In [4]:
h_id = pd.read_csv('../data/horse_ids.csv')
h_id.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14916 entries, 0 to 14915
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Unnamed: 0       14916 non-null  int64 
 1   track_id         14916 non-null  object
 2   race_date        14916 non-null  object
 3   race             14916 non-null  int64 
 4   program_number   14916 non-null  object
 5   horse_id         14916 non-null  int64 
 6   finishing_place  14916 non-null  int64 
dtypes: int64(4), object(3)
memory usage: 815.8+ KB


In [5]:
h_names = pd.read_csv('../data/horse_names.csv')
h_names.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4638 entries, 0 to 4637
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  4638 non-null   int64 
 1   horse_id    4638 non-null   int64 
 2   horse_name  4638 non-null   object
dtypes: int64(2), object(1)
memory usage: 108.8+ KB


In [6]:
tracking = pd.read_csv('../data/nyra_tracking_table.csv')
tracking.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5228430 entries, 0 to 5228429
Data columns (total 7 columns):
 #   Column          Dtype  
---  ------          -----  
 0   track_id        object 
 1   race_date       object 
 2   race_number     int64  
 3   program_number  object 
 4   trakus_index    int64  
 5   latitude        float64
 6   longitude       float64
dtypes: float64(2), int64(2), object(3)
memory usage: 279.2+ MB


##### Merging all data sets

In [7]:
# merging Horse ID and Horse Name data sets
h_id = h_id[['track_id', 'race_date', 'race', 'program_number', 'horse_id', 'finishing_place']]
h_names = h_names[['horse_id', 'horse_name']]
horses = pd.merge(h_id, h_names, how='left', on='horse_id')
horses = horses.rename(columns = {'race' : 'race_number'})

# Adding in Starts data
starts['race_date'] = pd.to_datetime(starts['race_date'])
horses['race_date'] = pd.to_datetime(horses['race_date'])
# strip spaces in the starts program_number column
starts['program_number'] = starts['program_number'].str.strip()
# merge
st_h = pd.merge(starts, horses, how='outer', on=['track_id', 'race_date', 'race_number', 'program_number'])
st_h = st_h.dropna(subset='odds')

# Adding in race details
races['race_date'] = pd.to_datetime(races['race_date'])
ra_st_h = pd.merge(st_h, races, how='left', on=['track_id', 'race_date', 'race_number'])

# Adding in tracking data
tracking['program_number'] = tracking['program_number'].str.strip()
tracking['race_date'] = pd.to_datetime(tracking['race_date'])
full = pd.merge(ra_st_h, tracking, how='left', on=['track_id', 'race_date', 'race_number', 'program_number'])

#Because races that include jumps are so drastically different than flat races, and such a comparatively small proportion of races
#we decided to drop those from our data set for the majority of our analysis
full = full.loc[full['course_type'] != 'M']

full = full.sort_values(['track_id', 'race_date', 'race_number', 'program_number', 'trakus_index']).reset_index(drop=True)

full.head()

,track_id,race_date,race_number,program_number,weight_carried,jockey,odds,horse_id,finishing_place,horse_name,distance_id,course_type,track_condition,run_up_distance,race_type,purse,post_time,trakus_index,latitude,longitude
0,AQU,2019-01-01,1,1,123.0,Dylan Davis,130.0,1,2,Sounds Delicious,650,D,MY,36,AOC,80000,1220,1,40.669401,-73.829205
1,AQU,2019-01-01,1,1,123.0,Dylan Davis,130.0,1,2,Sounds Delicious,650,D,MY,36,AOC,80000,1220,2,40.669405,-73.829203
2,AQU,2019-01-01,1,1,123.0,Dylan Davis,130.0,1,2,Sounds Delicious,650,D,MY,36,AOC,80000,1220,3,40.669411,-73.829200
3,AQU,2019-01-01,1,1,123.0,Dylan Davis,130.0,1,2,Sounds Delicious,650,D,MY,36,AOC,80000,1220,4,40.669421,-73.829196
4,AQU,2019-01-01,1,1,123.0,Dylan Davis,130.0,1,2,Sounds Delicious,650,D,MY,36,AOC,80000,1220,5,40.669433,-73.829190


In [8]:
full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5162881 entries, 0 to 5162880
Data columns (total 20 columns):
 #   Column           Dtype         
---  ------           -----         
 0   track_id         object        
 1   race_date        datetime64[ns]
 2   race_number      int64         
 3   program_number   object        
 4   weight_carried   float64       
 5   jockey           object        
 6   odds             float64       
 7   horse_id         int64         
 8   finishing_place  int64         
 9   horse_name       object        
 10  distance_id      int64         
 11  course_type      object        
 12  track_condition  object        
 13  run_up_distance  int64         
 14  race_type        object        
 15  purse            int64         
 16  post_time        int64         
 17  trakus_index     int64         
 18  latitude         float64       
 19  longitude        float64       
dtypes: datetime64[ns](1), float64(4), int64(8), object(7)
memory usage: 

In [11]:
#taking a look at the output of the geodesic function
#using feet because horse races are measured in fractions of a mile
geodesic([full['latitude'][0], full['longitude'][0]], [full['latitude'][1], full['longitude'][1]]).feet

1.7347089642202558

In [8]:
#adding columns that give the next lat and lon for each horse
full['n_lat'] = full.groupby(['track_id', 'race_date', 'race_number', 
                              'program_number'])['latitude'].shift(-1)
full['n_lon'] = full.groupby(['track_id', 'race_date', 'race_number', 
                              'program_number'])['longitude'].shift(-1)
full

,track_id,race_date,race_number,program_number,weight_carried,jockey,odds,horse_id,finishing_place,horse_name,...,track_condition,run_up_distance,race_type,purse,post_time,trakus_index,latitude,longitude,n_lat,n_lon
0,AQU,2019-01-01,1,1,123.0,Dylan Davis,130.0,1,2,Sounds Delicious,...,MY,36,AOC,80000,1220,1,40.669401,-73.829205,40.669405,-73.829203
1,AQU,2019-01-01,1,1,123.0,Dylan Davis,130.0,1,2,Sounds Delicious,...,MY,36,AOC,80000,1220,2,40.669405,-73.829203,40.669411,-73.829200
2,AQU,2019-01-01,1,1,123.0,Dylan Davis,130.0,1,2,Sounds Delicious,...,MY,36,AOC,80000,1220,3,40.669411,-73.829200,40.669421,-73.829196
3,AQU,2019-01-01,1,1,123.0,Dylan Davis,130.0,1,2,Sounds Delicious,...,MY,36,AOC,80000,1220,4,40.669421,-73.829196,40.669433,-73.829190
4,AQU,2019-01-01,1,1,123.0,Dylan Davis,130.0,1,2,Sounds Delicious,...,MY,36,AOC,80000,1220,5,40.669433,-73.829190,40.669448,-73.829184
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5162876,SAR,2019-09-02,11,8,119.0,Luis Saez,110.0,2295,6,Mine the Coin,...,SY,32,MCL,40000,614,329,43.071944,-73.771727,43.071933,-73.771760
5162877,SAR,2019-09-02,11,8,119.0,Luis Saez,110.0,2295,6,Mine the Coin,...,SY,32,MCL,40000,614,330,43.071933,-73.771760,43.071922,-73.771792
5162878,SAR,2019-09-02,11,8,119.0,Luis Saez,110.0,2295,6,Mine the Coin,...,SY,32,MCL,40000,614,331,43.071922,-73.771792,43.071911,-73.771825
5162879,SAR,2019-09-02,11,8,119.0,Luis Saez,110.0,2295,6,Mine the Coin,...,SY,32,MCL,40000,614,332,43.071911,-73.771825,43.071899,-73.771858


In [12]:
full['seg_dist_ft'] = [geodesic([full['latitude'][i], full['longitude'][i]], 
                                              [full['n_lat'][i], full['n_lon'][i]]).feet
                       if pd.isna(full['n_lat'][i]) == False else float('nan') for i in range(len(full))] 

In [13]:
full

,track_id,race_date,race_number,program_number,weight_carried,jockey,odds,horse_id,finishing_place,horse_name,...,run_up_distance,race_type,purse,post_time,trakus_index,latitude,longitude,n_lat,n_lon,seg_dist_ft
0,AQU,2019-01-01,1,1,123.0,Dylan Davis,130.0,1,2,Sounds Delicious,...,36,AOC,80000,1220,1,40.669401,-73.829205,40.669405,-73.829203,1.734709
1,AQU,2019-01-01,1,1,123.0,Dylan Davis,130.0,1,2,Sounds Delicious,...,36,AOC,80000,1220,2,40.669405,-73.829203,40.669411,-73.829200,2.408953
2,AQU,2019-01-01,1,1,123.0,Dylan Davis,130.0,1,2,Sounds Delicious,...,36,AOC,80000,1220,3,40.669411,-73.829200,40.669421,-73.829196,3.639469
3,AQU,2019-01-01,1,1,123.0,Dylan Davis,130.0,1,2,Sounds Delicious,...,36,AOC,80000,1220,4,40.669421,-73.829196,40.669433,-73.829190,4.522462
4,AQU,2019-01-01,1,1,123.0,Dylan Davis,130.0,1,2,Sounds Delicious,...,36,AOC,80000,1220,5,40.669433,-73.829190,40.669448,-73.829184,5.894653
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5162876,SAR,2019-09-02,11,8,119.0,Luis Saez,110.0,2295,6,Mine the Coin,...,32,MCL,40000,614,329,43.071944,-73.771727,43.071933,-73.771760,9.501454
5162877,SAR,2019-09-02,11,8,119.0,Luis Saez,110.0,2295,6,Mine the Coin,...,32,MCL,40000,614,330,43.071933,-73.771760,43.071922,-73.771792,9.609339
5162878,SAR,2019-09-02,11,8,119.0,Luis Saez,110.0,2295,6,Mine the Coin,...,32,MCL,40000,614,331,43.071922,-73.771792,43.071911,-73.771825,9.743090
5162879,SAR,2019-09-02,11,8,119.0,Luis Saez,110.0,2295,6,Mine the Coin,...,32,MCL,40000,614,332,43.071911,-73.771825,43.071899,-73.771858,9.565229


In [14]:
full['seg_speed_ft/s'] = full['seg_dist_ft']*4

full['seg_mph'] = 0.6818181818 * full['seg_speed_ft/s']

full['cum_dist_ft'] = full.groupby(['track_id', 'race_date', 'race_number', 
                              'program_number'])['seg_dist_ft'].cumsum()

full['avg_speed_ft/s'] = full['cum_dist_ft'] / (full['trakus_index']/4)

full['avg_mph'] = 0.6818181818 * full['avg_speed_ft/s']

full['seg_accel_ft/s2'] = 4 * full['seg_speed_ft/s'].diff()

full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5162881 entries, 0 to 5162880
Data columns (total 29 columns):
 #   Column           Dtype         
---  ------           -----         
 0   track_id         object        
 1   race_date        datetime64[ns]
 2   race_number      int64         
 3   program_number   object        
 4   weight_carried   float64       
 5   jockey           object        
 6   odds             float64       
 7   horse_id         int64         
 8   finishing_place  int64         
 9   horse_name       object        
 10  distance_id      int64         
 11  course_type      object        
 12  track_condition  object        
 13  run_up_distance  int64         
 14  race_type        object        
 15  purse            int64         
 16  post_time        int64         
 17  trakus_index     int64         
 18  latitude         float64       
 19  longitude        float64       
 20  n_lat            float64       
 21  n_lon            float64       

In [16]:
full = full[['track_id', 'race_date', 'race_number', 'program_number', 'horse_id', 'weight_carried', 
                'jockey', 'odds', 'finishing_place', 'horse_name', 'course_type', 'track_condition',
                'race_type', 'purse', 'post_time', 'latitude', 'longitude', 'n_lat', 'n_lon', 'distance_id', 
                'run_up_distance', 'trakus_index', 'seg_dist_ft', 'seg_speed_ft/s', 'seg_mph', 'cum_dist_ft', 
                'avg_speed_ft/s', 'avg_mph', 'seg_accel_ft/s2']]

In [17]:
import pickle

with open('../data/full_race_data.pickle', 'wb') as file:
    pickle.dump(full, file)

In [18]:
full.head(50)

,track_id,race_date,race_number,program_number,horse_id,weight_carried,jockey,odds,finishing_place,horse_name,...,distance_id,run_up_distance,trakus_index,seg_dist_ft,seg_speed_ft/s,seg_mph,cum_dist_ft,avg_speed_ft/s,avg_mph,seg_accel_ft/s2
0,AQU,2019-01-01,1,1,1,123.0,Dylan Davis,130.0,2,Sounds Delicious,...,650,36,1,1.734709,6.938836,4.731024,1.734709,6.938836,4.731024,NaN
1,AQU,2019-01-01,1,1,1,123.0,Dylan Davis,130.0,2,Sounds Delicious,...,650,36,2,2.408953,9.635811,6.569871,4.143662,8.287323,5.650448,10.787900
2,AQU,2019-01-01,1,1,1,123.0,Dylan Davis,130.0,2,Sounds Delicious,...,650,36,3,3.639469,14.557874,9.925823,7.783130,10.377507,7.075573,19.688254
3,AQU,2019-01-01,1,1,1,123.0,Dylan Davis,130.0,2,Sounds Delicious,...,650,36,4,4.522462,18.089850,12.333988,12.305593,12.305593,8.390177,14.127902
4,AQU,2019-01-01,1,1,1,123.0,Dylan Davis,130.0,2,Sounds Delicious,...,650,36,5,5.894653,23.578613,16.076327,18.200246,14.560197,9.927407,21.955053
5,AQU,2019-01-01,1,1,1,123.0,Dylan Davis,130.0,2,Sounds Delicious,...,650,36,6,6.820300,27.281199,18.600817,25.020546,16.680364,11.372975,14.810343
6,AQU,2019-01-01,1,1,1,123.0,Dylan Davis,130.0,2,Sounds Delicious,...,650,36,7,8.303640,33.214559,22.646290,33.324185,19.042392,12.983449,23.733441
7,AQU,2019-01-01,1,1,1,123.0,Dylan Davis,130.0,2,Sounds Delicious,...,650,36,8,9.376644,37.506575,25.572665,42.700829,21.350414,14.557101,17.168064
8,AQU,2019-01-01,1,1,1,123.0,Dylan Davis,130.0,2,Sounds Delicious,...,650,36,9,9.459504,37.838018,25.798649,52.160333,23.182370,15.806162,1.325773
9,AQU,2019-01-01,1,1,1,123.0,Dylan Davis,130.0,2,Sounds Delicious,...,650,36,10,10.503063,42.012251,28.644716,62.663396,25.065358,17.090017,16.696931
